# Image Labeler

En este notebook pretendemos crear una widget que nos permita etiquetar los fitolitos.

In [7]:
from __future__ import print_function
#Imports
%matplotlib inline
import numpy as np

In [8]:
import ipywidgets as widgets
from traitlets import Unicode, validate, Bool, Any, List, observe
from ipywidgets import Color
from skimage import io
from skimage.transform import resize

class ImageLabelerWidget(widgets.DOMWidget):
    _view_name = Unicode('ImageLabelerView').tag(sync=True)
    _view_module = Unicode('ImageLabeler').tag(sync=True)
    # Atributos sincronos entre js y python
    image = Unicode('blank.png').tag(sync=True)
    rect = List().tag(sync=True)
    
    Phytolith_type = Any().tag(sync=True)
    
    def __init__(self, *pargs, **kwargs):
        super(widgets.DOMWidget, self).__init__(*pargs, **kwargs)
        self._rectList = []
        
    @observe('rect')
    def new_rect(self, change):
        rec = [round(coord) for coord in change["new"]]
        self._rectList.append(rec)
        self.save_coords_as_images()
        
    def save_coords_as_images(self):
        image_string = image_labeler.image.title()
        image_data = io.imread(image_string)
        # Rescalamos imagen dependiendo del tamaño
        # en función del ancho o del alto
        h, w, _ = image_data.shape
        padding_h, padding_w = 0, 0
        # TODO Cuidado con si es menor de  400 x 400
        if h > w:
            coef = h/400
            h = 400
            w = round(w / coef)
        else:
            coef = w/400
            w = 400
            h = round(h / coef)
        padding_h = (400 - h) / 2
        padding_w = (400 - w) / 2
        print('h ', h,' ,w ', w, 'pad_h', padding_h, 'pad_w', padding_w)
        image_data = resize(image_data,(h,w))
        # Por cada rectangulo generamos una nueva imagen
        for rect_y, rect_x, rect_width, rect_height in self._rectList:
            rect_x = rect_x - round(padding_h)
            rect_y = rect_y - round(padding_w)
            name = str(rect_x)+str(rect_y)+".jpg"
            print(rect_x,rect_x+rect_height,rect_y,rect_y+rect_width)
            img = image_data[rect_x:rect_x+rect_height,rect_y:rect_y+rect_width]
            io.imsave(name, img)

In [9]:
%%javascript
require.undef('ImageLabeler');

var startX, startY;
var count = 0;
var rect = null;
var rectsList = [];
var model;
var myThis;

function getMousePos(evt) {
    // Obtenemos posición absoluta del elemento svg
    var svgRef = document.getElementById("svg");
    var htmlPos = svgRef.getBoundingClientRect();
    return {
        x: evt.clientX - htmlPos.left,
        y: evt.clientY - htmlPos.top
    };
}

function updateRect(eve, rect){
    var pos = getMousePos(eve);
    var endX = pos.x;
    var endY = pos.y;
    if (endX - startX > 0){
        var x = startX
    }else{
        var x = endX;
    }

    if (endY - startY > 0){
        var y = startY
    }else{
        var y = endY;
    }
    var width = Math.abs(endX - startX);
    var height = Math.abs(endY - startY);
    
    rect.setAttributeNS(null, 'x', x);
    rect.setAttributeNS(null, 'y', y);
    rect.setAttributeNS(null, 'height', height);
    rect.setAttributeNS(null, 'width', width);
}

function setStyleRect(rect){
    rect.setAttributeNS(null, 'fill', 'transparent');
    rect.setAttributeNS(null, 'stroke', 'green');
    rect.setAttributeNS(null, 'linewidth', '10px');
}

function mouseMove(eve) {
    if(rect){
        updateRect(eve, rect);
      }
}

function mouseClick(eve) {
    if (rect !== null) {
        //Cada vez que finalizamos el rectangulo
        // , le añadimos a la lista
        console.log(rect.x['baseVal'].value,rect.y['baseVal'].value,rect.width['baseVal'].value,rect.height['baseVal'].value);
        rectsList = [];
        model.set('rect',[rect.x['baseVal'].value,rect.y['baseVal'].value,rect.width['baseVal'].value,rect.height['baseVal'].value]);
        myThis.touch();
        rect = null;
        console.log("finsihed.");
    } else {
        var pos = getMousePos(eve);
        
        startX= pos.x;
        startY = pos.y;
        
        console.log("begun.");
        
        if(!rect){
            rect = document.createElementNS("http://www.w3.org/2000/svg", 'rect');
            rect.id = 'rect' + count;
            count = count + 1;
            
            updateRect(eve, rect);

            setStyleRect(rect);
            document.getElementById('svg').appendChild(rect);
          }
        }
}

// Funciones para la carga y cambio de imagen
function loadImage(svg, src){
    svg.innerHTML = '<image id="image" xlink:href="'+ src +'" x="0" y="0" height="400px" width="400px"/>'
}

function changeImage(svg, el, image){
    console.log("Cambio de imagen");
    console.log(image);
    svg.innerHTML = '<image id="image" xlink:href="' + image + '" x="0" y="0" height="400px" width="400px"/>';
    el = svg
}


// Definición del Widget en javasript
define('ImageLabeler', ["jupyter-js-widgets"], function(widgets) {
    var svg;
    var ImageLabelerView = widgets.DOMWidgetView.extend({
        // Renderizar vista
        render: function() {
            model = this.model;
            myThis = this;
            // Creamos el SVG
            svg = document.createElementNS("http://www.w3.org/2000/svg", "svg");
            svg.id = 'svg';
            svg.setAttribute('width', '400px');
            svg.setAttribute('height', '400px');
            
            // Asignamos listeners al SVG
            svg.addEventListener("click", mouseClick, false);
            svg.addEventListener("mousemove", mouseMove, false);
            
            // Imágen
            var src = "../rsc/img/family.jpg";
            loadImage(svg, src);
            // Listener para cambios de imagen
            this.image_changed();
            this.model.on('change:image', this.image_changed, this);
            
            // Asignamos a la vista el elemento SVG
            this.el = svg
        },
        // Función que se encarga del cambio de imagen
         image_changed: function() {
             var image = this.model.get('image');
             changeImage(svg, this.el, image);
        },
    });

    return {
        ImageLabelerView: ImageLabelerView
    };
});

<IPython.core.display.Javascript object>

# Añadimos más widgets

En esta celda añadimos el *file upload*

In [10]:
from IPython.display import display
import fileupload
import os
import PIL.Image
import io as io2
from skimage.color import rgb2gray

''' Función que se encarga de aplicar las operaciones 
necesarias para convertir los datos obtenidos del FileUpload
en una imagen'''
def imageConverter(change):
        ch = change['owner']
        image = io2.BytesIO(ch.data)
        image = PIL.Image.open(image)
        image = np.array(image)
        # Además, eliminamos la anterior 
        # y la guardamos
        rand = np.random.randint(0,1e6)
        #os.remove(rand + ".jpg")
        io.imsave(str(rand) + ".jpg",image)
        return str(rand) + ".jpg"



### Inicializamos el resto de Widgets

In [11]:
from ipywidgets import HBox, VBox, Label, Layout

#  Inicializamos el label título
title = Label(value = 'Etiquetador de imágenes')
#title.layout.align_items
title.layout.margin = '0 0 0 35%'
print(title.layout)

#Inicializamos Widget de File Upload
upload_widget = fileupload.FileUploadWidget()
upload_widget.description = '(50% width, 80px height) button'

# Inicializamos Widget etiquetador de imágenes
image_labeler = ImageLabelerWidget()
# Callback para el cambio de imagen
def _cb(change):
    # Guardamos imagen
    image = imageConverter(change)
    # Sincronizamos cambio
    image_labeler.image = image

upload_widget.observe(_cb, names='data')

#
phytolithType1_bt = Label('1')
phytolithType2_bt = Label('2')

In [13]:
VBox([title,
      HBox([
          image_labeler,
            VBox([
                upload_widget,
                phytolithType1_bt,
                phytolithType2_bt
            ])
      ])
     ])